In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn import preprocessing,model_selection
import itertools
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from pandas import Series, DataFrame
from collections import Counter, defaultdict

In [2]:
data = pd.read_csv('cs-training.csv')
#test_df = pd.read_csv('cs-test.csv')

In [3]:
print("Shape for dataset is {}".format(data.shape))

Shape for dataset is (150000, 12)


In [4]:
print("Info for dataset")
data.info()

Info for dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
Unnamed: 0                              150000 non-null int64
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(8)
memory usage: 13.7 MB


In [5]:
## MonthlyIncome                           120269 non-null float64
## NumberOfDependents                      146076 non-null float64

In [6]:
data.head(20)

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
5,6,0,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0
6,7,0,0.305682,57,0,5710.000000,NaN,8,0,3,0,0.0
7,8,0,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0
8,9,0,0.116951,27,0,46.000000,NaN,2,0,0,0,NaN
9,10,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0


In [7]:
# выводим информацию об общем количестве пропущенных 
# наблюдений в обучающей и контрольной выборках
print(data.isnull().sum().sum())

33655


In [8]:
##    SeriousDlqin2yrs - наличие просрочек
##    RevolvingUtilizationOfUnsecuredLines - утилизация кредитной карты
##    NumberOfTime30-59DaysPastDueNotWorse - кол-во просрочек от 30 до 60 дней
##    DebtRatio - отношение долга к доходу
##    MonthlyIncome - ежемесячный доход
##    NumberOfOpenCreditLinesAndLoans - кол-во открытых кредитных линий
##    NumberOfTimes90DaysLate - кол-во просрочек 90+
##    NumberRealEstateLoansOrLines - кол-во кредитных линий по ипотеке
##    NumberOfTime60-89DaysPastDueNotWorse - кол-во просрочек от 60 до 90 дней
##    NumberOfDependents - кол-во иждивенцев

In [9]:
MeanMonthlyIncomeLess20years = data.loc[(data ['age'] < 20)][['MonthlyIncome']].mean()
MedianMonthlyIncomeLess20years = data.loc[(data ['age'] < 20)][['MonthlyIncome']].median()
MeanMonthlyIncome20_30ears = data.loc[(data ['age'] >= 20 ) & (data ['age'] < 30 )][['MonthlyIncome']].mean()
MeanMonthlyIncome30_40ears = data.loc[(data ['age'] >= 30 ) & (data ['age'] < 40 )][['MonthlyIncome']].mean()
MeanMonthlyIncome40_50ears = data.loc[(data ['age'] >= 40 ) & (data ['age'] < 50 )][['MonthlyIncome']].mean()
MeanMonthlyIncome50_60ears = data.loc[(data ['age'] >= 50 ) & (data ['age'] < 60 )][['MonthlyIncome']].mean()
MeanMonthlyIncome60_70years = data.loc[(data ['age'] >= 60 ) & (data ['age'] < 70 )][['MonthlyIncome']].mean()
MeanMonthlyIncomeMore70years = data.loc[(data ['age'] >= 70)][['MonthlyIncome']].mean()

print('MeanMonthlyIncomeLess20years = ', MeanMonthlyIncomeLess20years)
print('MedianMonthlyIncomeLess20years = ', MedianMonthlyIncomeLess20years)
print('MeanMonthlyIncome20_30ears = ', MeanMonthlyIncome20_30ears)
print('MeanMonthlyIncome30_40ears = ', MeanMonthlyIncome30_40ears)
print('MeanMonthlyIncome40_50ears = ', MeanMonthlyIncome40_50ears)
print('MeanMonthlyIncome50_60ears = ', MeanMonthlyIncome50_60ears)
print('MeanMonthlyIncome60_70years = ', MeanMonthlyIncome60_70years)
print('MeanMonthlyIncomeMore70years = ', MeanMonthlyIncomeMore70years)

MeanMonthlyIncomeLess20years =  MonthlyIncome    6000.0
dtype: float64
MedianMonthlyIncomeLess20years =  MonthlyIncome    6000.0
dtype: float64
MeanMonthlyIncome20_30ears =  MonthlyIncome    2988.597723
dtype: float64
MeanMonthlyIncome30_40ears =  MonthlyIncome    5520.177019
dtype: float64
MeanMonthlyIncome40_50ears =  MonthlyIncome    7219.75602
dtype: float64
MeanMonthlyIncome50_60ears =  MonthlyIncome    7776.737808
dtype: float64
MeanMonthlyIncome60_70years =  MonthlyIncome    7433.951819
dtype: float64
MeanMonthlyIncomeMore70years =  MonthlyIncome    5642.657488
dtype: float64


In [10]:
#NumberOfDependents
NumberOfDependentsLess20years = data.loc[(data ['age'] < 20)][['NumberOfDependents']].mean()
NumberOfDependents20_30ears = data.loc[(data ['age'] >= 20 ) & (data ['age'] < 30 )][['NumberOfDependents']].mean()
NumberOfDependents30_40ears = data.loc[(data ['age'] >= 30 ) & (data ['age'] < 40 )][['NumberOfDependents']].mean()
NumberOfDependents40_50ears = data.loc[(data ['age'] >= 40 ) & (data ['age'] < 50 )][['NumberOfDependents']].mean()
NumberOfDependents50_60ears = data.loc[(data ['age'] >= 50 ) & (data ['age'] < 60 )][['NumberOfDependents']].mean()
NumberOfDependents60_70years = data.loc[(data ['age'] >= 60 ) & (data ['age'] < 70 )][['NumberOfDependents']].mean()
NumberOfDependentsMore70years = data.loc[(data ['age'] >= 70)][['NumberOfDependents']].mean()

print('NumberOfDependentsLess20years = ', NumberOfDependentsLess20years)
print('NumberOfDependents20_30ears = ', NumberOfDependents20_30ears)
print('NumberOfDependents30_40ears = ', NumberOfDependents30_40ears)
print('NumberOfDependents40_50ears = ', NumberOfDependents40_50ears)
print('NumberOfDependents50_60ears = ', NumberOfDependents50_60ears)
print('NumberOfDependents60_70years = ', NumberOfDependents60_70years)
print('NumberOfDependentsMore70years = ', NumberOfDependentsMore70years)

NumberOfDependentsLess20years =  NumberOfDependents    2.0
dtype: float64
NumberOfDependents20_30ears =  NumberOfDependents    0.293748
dtype: float64
NumberOfDependents30_40ears =  NumberOfDependents    0.928793
dtype: float64
NumberOfDependents40_50ears =  NumberOfDependents    1.30316
dtype: float64
NumberOfDependents50_60ears =  NumberOfDependents    0.849615
dtype: float64
NumberOfDependents60_70years =  NumberOfDependents    0.363403
dtype: float64
NumberOfDependentsMore70years =  NumberOfDependents    0.16572
dtype: float64


In [11]:
col_names = data.columns.values
col_names[0] = 'ID' ## rename first column to ID
data.columns = col_names ## assign new column name to training dataset

In [12]:
# удаляем переменную ID
data.drop('ID', axis=1, inplace=True)

In [13]:
data.head(20)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
5,0,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0
6,0,0.305682,57,0,5710.000000,NaN,8,0,3,0,0.0
7,0,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0
8,0,0.116951,27,0,46.000000,NaN,2,0,0,0,NaN
9,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0


In [14]:
# разбиваем данные на обучающие и тестовые
train, test, y_train, y_test = train_test_split(data.drop('SeriousDlqin2yrs', axis=1), \
                                                data['SeriousDlqin2yrs'], test_size=.3, \
                                                stratify=data['SeriousDlqin2yrs'], random_state=100)
train.head(10)

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
49898,0.719619,78,0,0.851277,3993.0,19,0,2,0,0.0
73629,0.470176,49,0,0.525397,8937.0,24,0,2,0,0.0
142688,0.140701,69,0,233.000000,NaN,10,0,0,0,0.0
112088,0.055935,51,0,837.000000,NaN,7,0,1,0,0.0
34526,0.009316,83,0,0.002940,5101.0,7,0,0,0,0.0
75695,0.020310,57,0,0.365728,14600.0,6,0,2,0,0.0
124944,0.000502,38,0,0.173270,3583.0,10,0,0,0,0.0
79452,0.665133,47,4,0.324540,8377.0,5,0,1,0,1.0
54915,0.114428,40,0,510.000000,NaN,3,0,0,0,0.0
49828,0.107926,57,1,0.363651,12566.0,14,0,2,0,2.0


In [15]:
# создаем экземпляр класса SimpleImputer
from sklearn.impute import SimpleImputer
simp = SimpleImputer(strategy='median')

# обучаем модель импутации - вычисляем медиану в обучающем массиве
simp.fit(train[['MonthlyIncome']])
simp.fit(train[['NumberOfDependents']])

train['MonthlyIncome'] = simp.transform(train[['MonthlyIncome']])
test['MonthlyIncome'] = simp.transform(test[['MonthlyIncome']])

train['NumberOfDependents'] = simp.transform(train[['NumberOfDependents']])
test['NumberOfDependents'] = simp.transform(test[['NumberOfDependents']])

In [16]:
print(train.isnull().sum())
print('')
print(test.isnull().sum())

RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64


In [17]:
#PIPELINE
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline

In [18]:
# создаем конвейер - экземпляр класса Pipeline
pipe = Pipeline([('scaler', StandardScaler()), \
                 ('logreg', LogisticRegression(solver='lbfgs', max_iter=200))])
pipe_quick = make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter=200))
pipe_quick.steps

[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('logisticregression',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=200,
                     multi_class='warn', n_jobs=None, penalty='l2',
                     random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                     warm_start=False))]

In [19]:
# обучаем конвейер
pipe.fit(train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logreg',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=200,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [20]:
# оцениваем качество модели на обучающих данных
print('Правильность на обучающей выборке: {:.3f}'.format(
pipe.score(train, y_train)))
# оцениваем качество модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
pipe.score(test, y_test)))

Правильность на обучающей выборке: 0.934
Правильность на тестовой выборке: 0.933


In [21]:
#Класс ColumnTransformer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('cs-training.csv')

# удаляем переменную ID
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head(5)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [22]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('SeriousDlqin2yrs', axis=1), \
                                                    data['SeriousDlqin2yrs'], test_size=0.3, \
                                                    stratify=data['SeriousDlqin2yrs'],random_state=42)

In [23]:
# создаем списки количественных
# и категориальных столбцов
cat_columns = X_train.dtypes[X_train.dtypes == 'object'].index
num_columns = X_train.dtypes[X_train.dtypes != 'object'].index
# создаем конвейер для количественных переменных
num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')), \
                     ('scaler', StandardScaler())])
# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), \
                     ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])
# создаем список трехэлементных кортежей
transformers = [('num', num_pipe, num_columns), ('cat', cat_pipe, cat_columns)]
transformer = ColumnTransformer(transformers=transformers)
# задаем итоговый конвейер
ml_pipe = Pipeline([('transform', transformer), ('logreg', LogisticRegression(solver='lbfgs',\
                                                                              max_iter=200))])
# обучаем итоговый конвейер
ml_pipe.fit(X_train, y_train)
# оцениваем качество модели на обучающих данных
print('Правильность на обучающей выборке: {:.3f}'.format(ml_pipe.score(X_train, y_train)))
# оцениваем качество модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
ml_pipe.score(X_test, y_test)))

Правильность на обучающей выборке: 0.934
Правильность на тестовой выборке: 0.934


In [24]:
#Функция cross_val_score()
# импортируем необходимые библиотеки, функцию train_test_split()
# и классы SimpleImputer, StandardScaler, OneHotEncoder, 
# ColumnTransformer, LogisticRegression, Pipeline,
# функцию cross_val_score()
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
data.head(5)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [25]:
print("Info for dataset")
data.info()

Info for dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(7)
memory usage: 12.6 MB


In [26]:
# создаем массив меток и массив признаков
y = data.pop('SeriousDlqin2yrs')
# создаем списки количественных и категориальных столбцов
cat_columns = data.dtypes[data.dtypes == 'object'].index
num_columns = data.dtypes[data.dtypes != 'object'].index

In [27]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')), \
                     ('scaler', StandardScaler())])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),\
                     ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [28]:
# создаем список трехэлементных кортежей
transformers = [('num', num_pipe, num_columns),('cat', cat_pipe, cat_columns)]
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)
# задаем итоговый конвейер
ml_pipe = Pipeline([('transform', transformer), \
                    ('logreg', LogisticRegression(solver='lbfgs', max_iter=200))])

In [29]:
# выполняем перекрестную проверку, разместив итоговый 
# конвейер внутри цикла перекрестной проверки
cv_mean = cross_val_score(ml_pipe, data, y, cv=5).mean()
print('Правильность, усредненная по 5 проверочным блокам cv: {:.3f}'.format(cv_mean))

Правильность, усредненная по 5 проверочным блокам cv: 0.934


In [30]:
#Класс GridSearchCV
# импортируем необходимые библиотеки, функции и классы
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
data = pd.read_csv('cs-training.csv')

# удаляем переменную ID
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head(5)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [31]:
print("Info for dataset")
data.info()

Info for dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(7)
memory usage: 12.6 MB


In [32]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('SeriousDlqin2yrs', axis=1), \
                                                    data['SeriousDlqin2yrs'], test_size=0.3, \
                                                    stratify=data['SeriousDlqin2yrs'], random_state=42)


# создаем конвейер для количественных переменных
num_pipe = Pipeline([('imp', SimpleImputer()), \
                     ('scaler', StandardScaler())])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([('imp', SimpleImputer()), \
                     ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [33]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns), 
                ('cat', cat_pipe, cat_columns)]

# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
ml_pipe = Pipeline([('tf', transformer), 
                    ('logreg', LogisticRegression(solver='lbfgs', max_iter=200))])

In [34]:
# задаем сетку гиперпараметров
param_grid = {'tf__num__imp__strategy': ['mean', 'median', 'constant'], \
              'tf__cat__imp__strategy': ['most_frequent', 'constant'], \
              'logreg__C': [.01, .1, .5, 1, 5, 10, 100]
}
# создаем экземпляр класса GridSearchCV
gs = GridSearchCV(ml_pipe, param_grid, cv=5, return_train_score=False)
# выполняем решетчатый поиск
gs.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# смотрим наилучшее значение правильности
print('Наилучшее значение правильности: {:.3f}'.format(gs.best_score_))
# смотрим значение правильности
# на тестовой выборке
print('Значение правильности на тестовой выборке: {:.3f}'.format(gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'logreg__C': 100, 'tf__cat__imp__strategy': 'most_frequent', 'tf__num__imp__strategy': 'mean'}
Наилучшее значение правильности: 0.934
Значение правильности на тестовой выборке: 0.934


In [35]:
# запишем результаты перекрестной проверки в DataFrame
results = pd.DataFrame(gs.cv_results_)
# превращаем в сводную таблицу
table = results.pivot_table(values=['mean_test_score'],    
                            index=['param_logreg__C', 
                                   'param_tf__cat__imp__strategy',
                                   'param_tf__num__imp__strategy'])
print(table)

                                                                           mean_test_score
param_logreg__C param_tf__cat__imp__strategy param_tf__num__imp__strategy                 
0.01            constant                     constant                             0.933305
                                             mean                                 0.933295
                                             median                               0.933305
                most_frequent                constant                             0.933305
                                             mean                                 0.933295
                                             median                               0.933305
0.10            constant                     constant                             0.933800
                                             mean                                 0.933829
                                             median                               0.933810

In [36]:
#KFold
# записываем CSV-файл в объект DataFrame
data2 = pd.read_csv('cs-training.csv', encoding='cp1251')
data2.head(5)


,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [37]:
# удаляем переменную ID
data2.drop('Unnamed: 0', axis=1, inplace=True)
data2.head(5)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [38]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
tr, tst, y_tr, y_tst = train_test_split(data2.drop('SeriousDlqin2yrs', axis=1), \
                                        data2['SeriousDlqin2yrs'], \
                                        test_size=.3, \
                                        stratify=data2['SeriousDlqin2yrs'], \
                                        random_state=100)

In [39]:
# создаем списки количественных 
# и категориальных столбцов
categorical_features = tr.dtypes[tr.dtypes == 'object'].index
numeric_features = tr.dtypes[tr.dtypes != 'object'].index

# создаем трансформеры
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')), \
                                          ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

# передаем список трансформеров в ColumnTransformer
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features), \
                                               ('cat', categorical_transformer, categorical_features)])

# формируем итоговый конвейер
pipe2 = Pipeline([('preprocessor', preprocessor), \
                  ('classifier', LogisticRegression(solver='lbfgs', max_iter=400))])

In [40]:
# задаем сетку гиперпараметров
param_grid2 = [{'classifier': [GradientBoostingClassifier(n_estimators=50, \
                                                          random_state=42, \
                                                          subsample=0.8)],\
                'classifier__max_depth': [4, 6, 8], \
                'preprocessor__num__scaler': [None]}, \
               {'classifier': [LogisticRegression(solver='lbfgs', max_iter=400)], \
                'classifier__C': [.05, .01], \
                'preprocessor__num__scaler': [None]}, \
               {'classifier': [LogisticRegression(solver='lbfgs', max_iter=400)], \
                'classifier__C': [.05, .01]}]

In [41]:
# создаем экземпляр класса KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров, оптимизируемую метрику, указав
# стратегию перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs2 = GridSearchCV(pipe2, param_grid2, scoring='roc_auc', cv=kf, return_train_score=False)
# выполняем решетчатый поиск
gs2.fit(tr, y_tr)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs2.best_params_))
# смотрим наилучшее значение AUC
print('Наилучшее значение AUC: {:.3f}'.format(gs2.best_score_))
# смотрим значение AUC на тестовой выборке
print('AUC на тестовом наборе: {:.3f}'.format(roc_auc_score(y_tst, gs2.predict_proba(tst)[:, 1])))

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarn

Наилучшие значения гиперпараметров: {'classifier': GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=0.8, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), 'classifier__max_depth': 6, 'preprocessor__num__scaler': None}
Наилучшее значение AUC: 0.863
AUC на тестовом наборе: 0.863


In [42]:
#Hyperopt
# импортируем необходимые библиотеки, классы и функции
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from numpy.random import RandomState

In [43]:
data3 = pd.read_csv('cs-training.csv', encoding='cp1251')
data3.head(5)

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [44]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
Unnamed: 0                              150000 non-null int64
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(8)
memory usage: 13.7 MB


In [45]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data3.drop('SeriousDlqin2yrs', axis=1), 
                                                    data3['SeriousDlqin2yrs'], 
                                                    test_size=0.3,
                                                    stratify=data3['SeriousDlqin2yrs'],
                                                    random_state=42)

In [46]:
# создаем списки количественных 
# и категориальных столбцов
cat_columns = X_train.dtypes[X_train.dtypes == 'object'].index
num_columns = X_train.dtypes[X_train.dtypes != 'object'].index

# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
pipe = Pipeline([('transform', transformer), 
                 ('logreg', LogisticRegression(solver='liblinear'))])

In [47]:
# пишем функцию, возвращающую правильность,
# усредненную по итогам перекрестной проверки
def hyperopt_objective(params):
    pipe.set_params(**params)
    return cross_val_score(pipe, X_train, y_train, cv=5).mean()

# создаем пространство значений гиперпараметров
space_hyperopt = {
    'logreg__C': hp.loguniform('logreg__C', np.log(0.01), np.log(100)),
    'transform__num__imputer__strategy': hp.choice(
        'transform__num__imputer__strategy', ['mean', 'median', 'constant']),
    'transform__cat__imputer__strategy': hp.choice(
        'transform__cat__imputer__strategy', ['most_frequent', 'constant'])
}

# пишем функцию, которая применяет ранее написанную
# нами функцию и возвращает результаты
def f(params):
    acc = hyperopt_objective(params)
    return {'loss': -acc, 'status': STATUS_OK}

# запускаем поиск
best = fmin(fn=f, 
            space=space_hyperopt,
            algo=tpe.suggest, 
            max_evals=50,
            rstate=RandomState(123))

# печатаем результаты
print('best:')
print(best)

100%|██████████| 50/50 [03:51<00:00,  4.49s/trial, best loss: -0.9339809604244899]
best:
{'logreg__C': 5.97477149456575, 'transform__cat__imputer__strategy': 1, 'transform__num__imputer__strategy': 0}


In [48]:
# записываем наилучшие значения гиперпараметров
best_params = space_eval(space_hyperopt, best)

# обучаем модель с наилучшими значениями
# гиперпараметров на обучающей выборке
pipe.set_params(**best_params)
pipe.fit(X_train, y_train)

# вычисляем прогнозы на тестовой выборке
test_predict = pipe.predict(X_test)

# вычисляем правильность для тестовой выборки
acc = accuracy_score(y_test, test_predict)
acc

0.9338444444444445

In [49]:
# импортируем класс BayesSearchCV, функцию gp_minimize()
from skopt import BayesSearchCV, gp_minimize
from skopt.space import Real, Categorical, Integer

In [56]:
# найдем такое значение x, которое минимизирует 
# квадратичную функцию y(x)=(x-1)**2
best = gp_minimize(
    func=lambda x: (x[0] - 1) ** 2,
    dimensions=[Real(-3, 3)],
    n_calls=35,
    random_state=123)

# печатаем результат
print("наилучшее значение = %.4f" % best.fun)
print('x = %.4f' %best.x[0])

наилучшее значение = 0.0000
x = 0.9999


In [57]:
# создаем пространство значений гиперпараметров
space_scopt = {'logreg__C': Real(np.log10(1.01), np.log10(100), 'log_uniform'),
               'transform__num__imputer__strategy': Categorical(['mean', 'median', 'constant']),
               'transform__cat__imputer__strategy': Categorical(['most_frequent', 'constant'])}

In [58]:
# fix pip install git+https://github.com/darenr/scikit-optimize
# создаем экземпляр класса BayesSearchCV
opt = BayesSearchCV(pipe, space_scopt, n_iter=10, random_state=42, cv=5)
# запускаем поиск
opt.fit(X_train, y_train)
# печатаем наилучшее значение правильности
print('наилучшее значение: %s' % opt.best_score_)

наилучшее значение: 0.9339428571428572


In [59]:
# сохраняем результаты в датафрейм
results = pd.DataFrame(opt.cv_results_)
# превращаем в сводную таблицу
tbl = results.pivot_table(values=['mean_test_score'],    
                          index=['param_logreg__C', 
                                 'param_transform__cat__imputer__strategy',
                                 'param_transform__num__imputer__strategy'])
print(tbl)

                                                                                                 mean_test_score
param_logreg__C param_transform__cat__imputer__strategy param_transform__num__imputer__strategy                 
0.004419        constant                                constant                                        0.933295
0.053545        constant                                constant                                        0.933629
0.066265        constant                                mean                                            0.933733
0.121342        constant                                median                                          0.933781
0.190717        constant                                median                                          0.933895
0.390936        constant                                mean                                            0.933905
0.584464        most_frequent                           median                                  

In [60]:
# смотрим наилучшее значение
print(opt.best_score_)
# смотрим наилучшие гиперпараметры
print(opt.best_params_)
# смотрим качество наилучшей модели на тестовой выборке
print('значение на тестовой выборке: %s' % opt.score(X_test, y_test))

0.9339428571428572
{'logreg__C': 0.6323954428081655, 'transform__cat__imputer__strategy': 'most_frequent', 'transform__num__imputer__strategy': 'median'}
значение на тестовой выборке: 0.9338222222222222


In [61]:
# записываем наилучшие значения гиперпараметров
best_params_scopt = opt.best_params_

# обучаем модель с наилучшими значениями
# гиперпараметров на обучающей выборке
pipe.set_params(**best_params_scopt)
pipe.fit(X_train, y_train)

# вычисляем прогнозы на тестовой выборке
test_predict = pipe.predict(X_test)

# вычисляем правильность для тестовой выборки
acc = accuracy_score(y_test, test_predict)
acc

0.9338222222222222